In [1]:
# from utils import * 

import os
os.chdir("../../scVI/")
os.getcwd()

import pickle
import numpy as np
import pandas as pd
# import gseapy as gp
from copy import deepcopy 
save_path = '../CSF/Notebooks/'

In [2]:
labels = np.load(save_path + 'meta/CD4.clusters.npy')
celltypes = np.unique(labels).astype(str)

In [3]:
celltypes

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'],
      dtype='<U21')

In [4]:
celllabels = np.load(save_path + 'meta/celllabels.npy')
isCD4only = celllabels=='CD4'

In [5]:
# from numpy import savetxt
# savetxt('../CSF/DE/raw/CD4labels.csv', labels, delimiter=',',fmt='%i')

In [6]:
# with open(save_path + 'posterior/CD4only.vae.full.pkl', 'rb') as f:
#     full = pickle.load(f)

# with open(save_path + 'posterior/all_datasets.vae.full.pkl', 'rb') as f:
#     full = pickle.load(f)


In [7]:
temp = np.repeat(-1,len(celllabels))
temp[isCD4only] = labels
# DEres, DEclust = full.one_vs_all_degenes(cell_labels=labels, output_file=False,save_dir=save_path, filename='LouvainClusters')
# DEres, DEclust = full.one_vs_all_degenes(cell_labels=temp, output_file=False,
#                                          save_dir=save_path, filename='CD4.clusters',
#                                         subset=isCD4)


In [8]:
# with open(save_path + 'DE/CD4only.DEres.pkl', 'wb') as f:
#     pickle.dump((DEres,DEclust),f)

In [9]:
with open(save_path + 'DE/CD4.DEres.pkl', 'rb') as f:
    DEres,DEclust = pickle.load(f)
    
genenames = pd.read_csv('../CSF/RFiles/genenames.csv',header=None)

In [10]:
from statsmodels.stats.multitest import multipletests

In [11]:
celltype = []
combinedDEres = []
for i,x in enumerate(celltypes[DEclust]):
    temp  = pd.read_csv('../CSF/DE/wilcoxon/MannWhitneyU.CD4.allclusters.%s.csv'%x)
    temp.index = list(genenames[0])
    fdr_wil = multipletests(temp['pvalue'],method='fdr_bh')
    temp = pd.concat([temp,DEres[i]],axis=1,sort=True)
    temp = temp.sort_values(by='bayes1',ascending=False)
    combined = deepcopy(temp[['stat','pvalue','bayes1','bayes2','mean1','mean2','nonz1','nonz2','clusters',]])
    combined['scVI_logFC'] = np.log(temp['scale1']/temp['scale2'])
    combined['norm_logFC'] = np.log(temp['norm_mean1']/temp['norm_mean2'])
    temp  = pd.read_csv('../CSF/EdgeR/CD4.batch_corrected.%s.edgeR.csv'%x)
    fdr_edgeR = multipletests(temp['PValue'],method='fdr_bh')
    temp.index = list(genenames[0])
    temp = temp[['logFC','logCPM','F','PValue']]
    combined = pd.concat([combined,temp],axis=1,sort=True)
    combined['fdr_wil'] = fdr_wil[1]
    combined['fdr_edgeR'] = fdr_edgeR[1]
    celltype.append(x)
    combinedDEres.append(combined)

    


/data/yosef2/users/chenling/miniconda3/envs/pytorch_new/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [14]:
# from pandas import ExcelWriter
# import xlsxwriter


# writer = pd.ExcelWriter(save_path + 'DE/allclusters.CD4.xlsx', engine='xlsxwriter')
# for i, x in enumerate(celltype):
#     combinedDEres[i].to_excel(writer, sheet_name=str(x))
# writer.close()


In [12]:
filtered = [x.loc[(x['fdr_wil']<0.2)&(x['fdr_edgeR']<0.2)& 
                  (x['bayes1']>0.3) &
                  (x['logFC']>0) & (x['norm_logFC']>0)] for x in combinedDEres]

DEgenes = pd.concat(filtered)

DEgenes['clusters'] = np.asarray(celltypes)[np.asarray(DEgenes['clusters']).astype(int)]
DEgenes.to_csv(save_path+'DE/CD4markers.relaxed.csv')

In [16]:
len(filtered)

12

In [20]:
combinedDEres[10].loc[['TNFRSF14','TIGIT','PDCD1','LAG3','HAVCR2','ICOS','CTLA4','TNFRSF9','LILRB4']]

,stat,pvalue,bayes1,bayes2,mean1,mean2,nonz1,nonz2,clusters,scVI_logFC,norm_logFC,logFC,logCPM,F,PValue,fdr_wil,fdr_edgeR
TNFRSF14,7004383.0,6.178923e-12,-0.927002,0.921648,0.178794,0.277818,0.162162,0.236395,10,-0.100004,-0.215803,-0.229936,9.442951,7.297386,1.641217e-02,4.111241e-51,0.212371
TIGIT,8889708.0,2.822103e-79,2.660324,-2.664606,0.286902,0.057789,0.205821,0.048571,10,0.947857,1.738618,1.341575,9.301205,751.397423,8.859154e-58,3.094868e-37,1.000000
PDCD1,8274260.0,1.744286e-50,0.338415,-0.335238,0.006237,0.017178,0.006237,0.016163,10,0.002758,-0.973237,-0.063399,9.269200,12.161986,2.346503e-01,7.187685e-33,0.597653
LAG3,8281963.0,8.358670e-51,0.590830,-0.586981,0.004158,0.013726,0.002079,0.012914,10,0.096698,-0.704830,-0.066514,9.266725,20.159916,1.655323e-01,3.849812e-53,0.981719
HAVCR2,8286444.0,5.442673e-51,-0.670532,0.673138,0.004158,0.014214,0.004158,0.013158,10,-0.162873,-0.953355,-0.060589,9.267020,17.054619,1.902414e-01,5.758623e-54,0.680546
ICOS,8131326.0,9.593016e-45,0.690773,-0.681445,0.051975,0.056327,0.051975,0.053119,10,0.140113,0.092362,0.063243,9.296964,1.463579,4.969950e-01,4.739911e-49,0.916396
CTLA4,8346279.0,1.637670e-53,1.350566,-1.345543,0.004158,0.007351,0.004158,0.006944,10,0.377319,-0.313706,-0.007172,9.262201,1.268082,7.826506e-01,4.615761e-02,0.007427
TNFRSF9,8363178.0,3.095619e-54,1.052157,-1.046373,0.008316,0.007513,0.008316,0.007391,10,0.191385,0.353570,0.025457,9.262354,4.856613,5.881740e-01,4.677154e-49,0.915627
LILRB4,8397259.0,1.038747e-55,0.893685,-0.899300,0.002079,0.001381,0.002079,0.001381,10,0.193852,0.392733,0.008721,9.257954,8.633532,6.902584e-01,6.908837e-42,0.924696


In [39]:
filtered = [x.loc[(x['fdr_wil']<0.05)&(x['fdr_edgeR']<0.05)& 
                  (x['bayes1']>0.5) &
                  (x['logFC']>0) & (x['norm_logFC']>0)] for x in combinedDEres]

DEgenes = pd.concat(filtered)

DEgenes['clusters'] = np.asarray(celltypes)[np.asarray(DEgenes['clusters']).astype(int)]
DEgenes.to_csv(save_path+'DE/CD4markers.csv')

In [44]:
[x for x in combinedDEres[0].index if x.startswith('CXCR')]

['CXCR3', 'CXCR4', 'CXCR5', 'CXCR6']

In [47]:
names,occ = np.unique(DEgenes.index,return_counts=True)

In [49]:
names[occ>3]

array(['CCDC65', 'CNBP', 'FAM45A', 'FBXW5', 'HIP1R', 'KIN', 'KLC1',
       'LEO1', 'MAD2L1', 'PDHB', 'PIGA', 'PITPNB', 'PPP1CB', 'PSMD5',
       'RMDN1', 'RPL7L1', 'RPS3', 'RUVBL1', 'SSR3', 'TRAP1', 'TXNDC15'],
      dtype=object)